In [1]:
import os
import numpy as np
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup
from selenium import webdriver
from IPython.core.display import display
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager


# PUBLIC TRANSPORTATION DATA SET DOWNLOAD
By launching this script, the https://www.citylines.co/data website is scraped and the last updated version of the transports json data sets are downloaded. A still-in-work part will allow to merge all the downloaded datasets to a more useful unique .csv file.

In [8]:
##########################################
### MODIFY THIS, to store the DataSet: ###
##########################################

localDataUrl = '/Volumes/Disk2/Courses MA3/MA3 - ADA/AIRBNB data/Transports'

##########################################
##########################################

url = 'https://www.citylines.co/data'

# get all the files names and the files urls
files_df = scrapeCityLines(url)
# download the data set
download_data(files_df)



Looking for [chromedriver 78.0.3904.105 mac64] driver in cache 
File found in cache by path [/Users/sirinesayagh/.wdm/drivers/chromedriver/78.0.3904.105/mac64/chromedriver]


In [4]:
def scrapeCityLines(url):
    # Selenium is able to simulate the browser, and so we can make it wait until 
    # the page finished loading before we are getting the data.
    driver   = webdriver.Chrome(ChromeDriverManager().install())
    response = driver.get(url)
    select   = Select(driver.find_element_by_class_name('c-field'))

    links_download = []
    files_names    = []
    idx            = 0

    for cities in select.options:

        # first element of the dropdown menu is Select City; this is not a city!!!

        if idx >= 1:
            opt   = cities.click()
            soup  = BeautifulSoup(driver.page_source)
            links = soup.find_all('a', class_ = 'c-link')

            for i in links:
                if i.has_attr('href') and ('/api/' in i['href']):
                    links_download.append('https://www.citylines.co' + i['href'])
                    files_names.append(i['download'])
        idx += 1
    
    driver.quit()

    files_df = pd.DataFrame({'file_name':files_names, 'link':links_download})
    
    return files_df

def download_data(files_df):
    # download all the dataset
    for index, row in files_df.iterrows():

            file_name = localDataUrl + '/' + row['file_name']
            print('Downloading... ' + str(index+1) +'/' + str(files_df.shape[0]) + ' : ' + file_name)

            urllib.request.urlretrieve(row['link'], file_name)

In [ ]:
# WORK in progress....
""""
from pandas.io.json import json_normalize 
import json

def mergeFiles(file_name, files_folder_path, main_node):
    
    
    all_files = os.listdir(files_folder_path)
    selected_files = [i for i in all_files if file_name in i]
    
    df_merged = pd.DataFrame()
    
    for fl in selected_files:
        print('Merging file... ' + fl)
        with open(files_folder_path + '/' + fl) as data_file:    
            data = json.load(data_file)
        df = pd.DataFrame()
    
        for i in data[main_node]:
            df = pd.concat([df, json_normalize(i)], sort = True)

        df = df.reset_index(drop = True)

        # further expand all left dict in columns
        cols_with_dict = []
        for j in df.columns:
            hasDict = "<class 'dict'>" in df[j].apply(lambda x: str(type(x[0])) if hasattr(x, '__len__') and len(x)> 0 else x).tolist()
            cols_with_dict.append(hasDict)

        for a, b in zip(cols_with_dict, df.columns):
            if a:
                dicts = [x[0] if len(x)>0 else {} for x in df[b]]
                df = pd.concat([df, json_normalize(records)], axis = 1, sort = True).drop(b, axis = 1)

        df['city'] = fl.split('_')[0]
        df.columns = [s.split('.')[-1] for s in df.columns.values]
        
        
        dfs = [df, df_merged]
        if df_merged.shape[0] > 0:
            df_merged = pd.concat(dfs, ignore_index=True)
        else:
            df_merged = df
        #df_merged = pd.concat([df, df_merged], sort = True, ignore_index = True)
        
    
    return df_merged



df_stations = mergeFiles('stations.geojson', files_folder_path = localDataUrl, main_node = 'features')
df_stations
"""

# Nested json files handling
Here, the Amsterdam nested json stations is converted to a pandas DataFrame. The goal will be to automatize this and do it for all the downloaded data sets. The above code is intended to do it but is still to debug. At the end the goal is to have an unique .csv file comprising all the stations for all the cities

In [9]:
#### TESTS
import json
from pandas.io.json import json_normalize 

with open('./Transports/amsterdam_stations.geojson') as data_file:    
            data = json.load(data_file)

df = pd.DataFrame()
for i in data['features']:
    df = pd.concat([df, json_normalize(i)], sort = True)
    
df = df.reset_index(drop = True)
    



cols_with_dict = []
for i in df.columns:
    hasDict = "<class 'dict'>" in df[i].apply(lambda x: str(type(x[0])) if hasattr(x, '__len__') and len(x)> 0 else x).tolist()
    cols_with_dict.append(hasDict)

for a, b in zip(cols_with_dict, df.columns):
    if a:
        dicts = [x[0] if len(x)>0 else {} for x in df[b]]
        df = pd.concat([df, json_normalize(dicts)], axis = 1, sort = True).drop(b, axis = 1)

df.columns = [s.split('.')[-1] for s in df.columns.values]  

df.to_csv('Amsterdam_stations.csv')

## FIND CITIES COMMUN TO BOTH DATASETS

In [28]:
# get cities from transport data set
all_transport_files = os.listdir("./Transports") 
transport_cities = [file.split('_')[0] for file in all_files]
transport_cities = list(set(transport_cities)) #drop duplicate because more than one file per city
transport_cities = [x.replace('-',' ') for x in transport_cities] #remove hyphen
transport_cities

['grenoble',
 'mulhouse',
 'savannah',
 'madrid',
 'los angeles',
 'besancon',
 'chicago',
 'innsbruck',
 'orleans',
 'le mans',
 'tours',
 'new york',
 'valencia',
 'dzaoudzi',
 'bilbao',
 'reims',
 'dallas',
 'guadalajara',
 'rennes',
 'edinburgh',
 'cincinnati',
 'osaka',
 'rosario',
 'ottawa',
 'lille',
 'beijing',
 'kuala lumpur',
 'valparaiso',
 'buenos aires',
 'dijon',
 'toulouse',
 'jakarta',
 'atlanta',
 'florianopolis',
 'sao paulo',
 'munich',
 'richmond',
 'boston',
 'rouen',
 'strasbourg',
 'amsterdam',
 'rome',
 'barcelona',
 'montreal',
 'lisbon',
 'genoa',
 'bordeaux',
 'newcastle on tyne',
 'marseilles',
 'shanghai',
 'graz',
 'tokyo',
 'prague',
 'nice',
 'la paz',
 'naples',
 'washington',
 'caracas',
 'san francisco',
 'bogota',
 'lima',
 'milan',
 'lyons',
 'hong kong',
 'mumbai',
 'paris',
 'mexico city',
 'le havre',
 'montpellier',
 'rio de janeiro',
 'budapest',
 'singapore',
 'sydney',
 'pune',
 'brussels',
 'portland united states',
 'brest',
 'berlin',
 've

In [27]:
# get cities from airbnb data set
all_airbnb_files = os.listdir("./data") 
airbnb_cities = [file.split('_')[1] for file in all_airbnb_files]
airbnb_cities = list(set(airbnb_cities)) #drop duplicate because more than one file per city
airbnb_cities = [x.lower() for x in airbnb_cities] #remove capital letters
airbnb_cities

['store',
 'washington, d.c.',
 'mallorca',
 'belize',
 'victoria',
 'mexico city',
 'edinburgh',
 'melbourne',
 'manchester',
 'jersey city',
 'beijing',
 'london',
 'san mateo county',
 'rio de janeiro',
 'broward county',
 'oslo',
 'venice',
 'girona',
 'vancouver',
 'porto',
 'quebec city',
 'trentino',
 'copenhagen',
 'oakland',
 'portland',
 'naples',
 'sevilla',
 'ottawa',
 'lyon',
 'los angeles',
 'istanbul',
 'munich',
 'paris',
 'toronto',
 'san diego',
 'malaga',
 'columbus',
 'barossa valley',
 'san francisco',
 'brussels',
 'barcelona',
 'thessaloniki',
 'bordeaux',
 'ghent',
 'tasmania',
 'boston',
 'barwon south west, vic',
 'cape town',
 'new york city',
 'florence',
 'cambridge',
 'geneva',
 'dublin',
 'western australia',
 'taipei',
 'milan',
 'lisbon',
 'salem, or',
 'ireland',
 'menorca',
 'stockholm',
 'berlin',
 'nashville',
 'denver',
 'valencia',
 'crete',
 'greater manchester',
 'madrid',
 'sicily',
 'sydney',
 'asheville',
 'bristol',
 'austin',
 'south aegean

In [29]:
commun_cities = list(set(transport_cities).intersection(set(airbnb_cities)))

In [30]:
commun_cities

['manchester',
 'beijing',
 'madrid',
 'vienna',
 'toronto',
 'tokyo',
 'prague',
 'naples',
 'chicago',
 'istanbul',
 'munich',
 'melbourne',
 'milan',
 'boston',
 'paris',
 'amsterdam',
 'san francisco',
 'buenos aires',
 'singapore',
 'stockholm',
 'valencia',
 'rome',
 'barcelona',
 'hong kong',
 'montreal',
 'los angeles',
 'london',
 'seattle',
 'rio de janeiro',
 'sydney',
 'lisbon',
 'edinburgh',
 'mexico city',
 'brussels',
 'berlin',
 'venice',
 'ottawa',
 'bordeaux']